# CNN Recommender System

# Prepare data section

In [ ]:
import pandas as pd
import torch
from tqdm import tqdm
import math
from urllib.request import urlretrieve
from zipfile import ZipFile
import os
import torch.nn as nn
import numpy as np
from math import sqrt
import random
from sklearn.model_selection import train_test_split
import warnings
import argparse
from collections import defaultdict

from time import time

import torch.optim as optim
from torch.autograd import Variable


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

%cd gdrive/MyDrive/Movielens_CNN
warnings.filterwarnings("ignore")

Mounted at /content/gdrive/
/content/gdrive/.shortcut-targets-by-id/1F6yIxxXssy71KTK3lpmYr1RQRUqw9PWU/Movielens_CNN


In [ ]:
# from evaluation import evaluate_ranking
from interactions import Interactions


# from evaluation import evaluate_ranking
from utils import *

import torch

import torch.nn as nn
import torch.nn.functional as F

from utils import activation_getter

### PARAMETERS

In [ ]:
# perturb_num = 10

In [ ]:
# path_to_model = "/content/gdrive/My Drive/data/model/random_ratings_data_{}/cnn_model.pt".format(perturb_num)
# path_to_txt = "/content/gdrive/My Drive/data/model/random_ratings_data_{}/cnn_evaluation.txt".format(perturb_num)
# path_to_data = "/content/gdrive/My Drive/data/"

# train_root = 'data/train_data{}.txt'.format(perturb_num)
# test_root = 'data/test_data{}.txt'.format(perturb_num)

path_to_model = "/content/gdrive/My Drive/data/model/ground_truth/cnn_model.pt"
path_to_txt = "/content/gdrive/My Drive/data/model/ground_truth/cnn_evaluation.txt"
path_to_data = "/content/gdrive/My Drive/data/"

train_root = 'data/train_data_ground_truth.txt'
test_root = 'data/test_data_ground_truth.txt'

In [ ]:
# max test length for k = 5, 10, 15
LEN_TEST_PER_USER = 15
# train length
L = 5
# predict length
T = 3

## Data

In [ ]:
df = pd.read_csv(path_to_data + 'truth.csv')
df = df[['user_id', 'movie_ids', 'ratings']]
df.rename(columns={"movie_ids": "sequence_movie_ids", "ratings": "sequence_ratings"}, inplace=True)

In [ ]:
# df = pd.read_csv(path_to_data + 'random_ratings_data_{}.csv'.format(perturb_num))
# df = df[['user_id', 'random_movie_ids', 'random_ratings']]
# df.rename(columns={"random_movie_ids": "sequence_movie_ids", "random_ratings": "sequence_ratings"}, inplace=True)

In [ ]:
# df = pd.read_csv(path_to_data + 'inputed_ratings_data_{}.csv'.format(perturb_num))
# df = df[['user_id', 'inp_movie_ids', 'inp_ratings']]
# df.rename(columns={"inp_movie_ids": "sequence_movie_ids", "inp_ratings": "sequence_ratings"}, inplace=True)

In [ ]:
from ast import literal_eval
df['sequence_movie_ids'] = df['sequence_movie_ids'].apply(lambda x: literal_eval(x))
df['sequence_ratings'] = df['sequence_ratings'].apply(lambda x: literal_eval(x))

In [ ]:
# remove users that have less than T + L + LEN_TEST_PER_USER
df['len'] = df['sequence_ratings'].apply(lambda x: len(x))
df = df[df['len'] >= T + L + LEN_TEST_PER_USER]
del df['len']

In [ ]:
df

,user_id,sequence_movie_ids,sequence_ratings
0,1,"[3186, 1721, 1270, 1022, 2340, 1836, 3408, 120...","[4, 4, 5, 5, 3, 5, 4, 4, 5, 4, 3, 5, 4, 4, 4, ..."
1,10,"[597, 858, 743, 1210, 1948, 2312, 3751, 1282, ...","[4, 3, 3, 4, 4, 5, 5, 5, 3, 3, 3, 5, 4, 4, 4, ..."
2,100,"[260, 1676, 1198, 541, 1210, 3948, 3536, 2567,...","[4, 3, 4, 3, 4, 3, 1, 1, 5, 4, 4, 3, 2, 3, 4, ..."
3,1000,"[971, 260, 2990, 2973, 1210, 3068, 3153, 1198,...","[4, 5, 4, 3, 5, 5, 2, 5, 5, 4, 5, 4, 3, 5, 5, ..."
4,1001,"[1198, 1617, 2885, 3909, 3555, 1479, 3903, 394...","[4, 4, 4, 2, 2, 1, 4, 5, 5, 4, 4, 4, 4, 3, 4, ..."
...,...,...,...
6035,995,"[1894, 260, 247, 433, 170, 74, 912, 3097, 1265...","[2, 4, 5, 3, 3, 4, 4, 4, 3, 5, 5, 5, 5, 5, 5, ..."
6036,996,"[1347, 2146, 1961, 2741, 1210, 527, 1196, 1213...","[4, 3, 5, 3, 5, 5, 5, 5, 4, 2, 5, 5, 5, 4, 5, ..."
6037,997,"[1196, 2082, 3247, 2447, 2633, 2028, 593, 318,...","[4, 3, 3, 3, 2, 5, 5, 5, 4, 4, 5, 4, 4, 3, 4, ..."
6038,998,"[2266, 1264, 1097, 1641, 805, 1388, 1968, 3751...","[3, 4, 5, 5, 4, 3, 4, 3, 4, 4, 4, 4, 5, 5, 4, ..."


# Turn dataframes to user, movie_id, rating, timestamp format

In [ ]:
df = df.set_index(['user_id']).apply(pd.Series.explode).reset_index()
df = df.reset_index()
df

,index,user_id,sequence_movie_ids,sequence_ratings
0,0,1,3186,4
1,1,1,1721,4
2,2,1,1270,5
3,3,1,1022,5
4,4,1,2340,3
...,...,...,...,...
994879,994879,999,1410,2
994880,994880,999,2269,3
994881,994881,999,2504,3
994882,994882,999,455,2


In [ ]:
test_data = df.groupby("user_id").tail(LEN_TEST_PER_USER)
train_data = df.drop(index=test_data["index"])

test_data = test_data.reset_index(drop=True)
del test_data["index"]
del train_data["index"]

In [ ]:
# remove rows that have a user or movie_id in test that is not in train
unique_users = list(train_data.user_id.unique())
unique_movies = list(train_data.sequence_movie_ids.unique())

# Defining the dict
dict_users = defaultdict(int)
dict_movies = defaultdict(int)
      
for user in unique_users:
    dict_users[user] = 1
for movie in unique_movies:
    dict_movies[movie] = 1

test_data = test_data[test_data['user_id'].isin(dict_users)]
test_data = test_data[test_data['sequence_movie_ids'].isin(dict_movies)]
print("New length of test dataframe: {}".format(test_data.shape[0]))

New length of test dataframe: 86780


In [ ]:
test_data

,user_id,sequence_movie_ids,sequence_ratings
0,1,594,4
1,1,2398,4
2,1,1545,4
3,1,527,5
4,1,745,3
...,...,...,...
86785,999,1410,2
86786,999,2269,3
86787,999,2504,3
86788,999,455,2


In [ ]:
train_data.to_csv(train_root, index=False, sep=" ",header=False)
test_data.to_csv(test_root, index=False, sep=" ",header=False)

# End Data Preprocessing

# CASER Module

In [ ]:
class Caser(nn.Module):
    """
    Convolutional Sequence Embedding Recommendation Model (Caser)[1].
    [1] Personalized Top-N Sequential Recommendation via Convolutional Sequence Embedding, Jiaxi Tang and Ke Wang , WSDM '18
    Parameters
    ----------
    num_users: int,
        Number of users.
    num_items: int,
        Number of items.
    model_args: args,
        Model-related arguments, like latent dimensions.
    """

    def __init__(self, num_users, num_items, model_args):
        super(Caser, self).__init__()
        self.args = model_args

        # init args
        L = self.args['L']
        dims = self.args['d']
        self.n_h = self.args['nh']
        self.n_v = self.args['nv']
        self.drop_ratio = self.args['drop']
        self.ac_conv = activation_getter[self.args['ac_conv']]
        self.ac_fc = activation_getter[self.args['ac_fc']]

        # user and item embeddings
        self.user_embeddings = nn.Embedding(num_users, dims)
        self.item_embeddings = nn.Embedding(num_items, dims)

        # vertical conv layer
        self.conv_v = nn.Conv2d(1, self.n_v, (L, 1))

        # horizontal conv layer
        lengths = [i + 1 for i in range(L)]
        self.conv_h = nn.ModuleList([nn.Conv2d(1, self.n_h, (i, dims)) for i in lengths])

        # fully-connected layer
        self.fc1_dim_v = self.n_v * dims
        self.fc1_dim_h = self.n_h * len(lengths)
        fc1_dim_in = self.fc1_dim_v + self.fc1_dim_h
        # W1, b1 can be encoded with nn.Linear
        self.fc1 = nn.Linear(fc1_dim_in, dims)
        # W2, b2 are encoded with nn.Embedding, as we don't need to compute scores for all items
        self.W2 = nn.Embedding(num_items, dims+dims)
        self.b2 = nn.Embedding(num_items, 1)

        # dropout
        self.dropout = nn.Dropout(self.drop_ratio)

        # weight initialization
        self.user_embeddings.weight.data.normal_(0, 1.0 / self.user_embeddings.embedding_dim)
        self.item_embeddings.weight.data.normal_(0, 1.0 / self.item_embeddings.embedding_dim)
        self.W2.weight.data.normal_(0, 1.0 / self.W2.embedding_dim)
        self.b2.weight.data.zero_()

        self.cache_x = None

    def forward(self, seq_var, user_var, item_var, for_pred=False):
        """
        The forward propagation used to get recommendation scores, given
        triplet (user, sequence, targets).
        Parameters
        ----------
        seq_var: torch.FloatTensor with size [batch_size, L]
            a batch of sequence
        user_var: torch.LongTensor with size [batch_size]
            a batch of user
        item_var: torch.LongTensor with size [batch_size, T]
            a batch of items
        for_pred: boolean, optional
            Train or Prediction. Set to True when evaluation.
        """

        # Embedding Look-up
        item_embs = self.item_embeddings(seq_var).unsqueeze(1)  # use unsqueeze() to get 4-D
        user_emb = self.user_embeddings(user_var).squeeze(1)

        # Convolutional Layers
        out, out_h, out_v = None, None, None
        # vertical conv layer
        if self.n_v:
            out_v = self.conv_v(item_embs)
            out_v = out_v.view(-1, self.fc1_dim_v)  # prepare for fully connect

        # horizontal conv layer
        out_hs = list()
        if self.n_h:
            for conv in self.conv_h:
                conv_out = self.ac_conv(conv(item_embs).squeeze(3))
                pool_out = F.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
                out_hs.append(pool_out)
            out_h = torch.cat(out_hs, 1)  # prepare for fully connect

        # Fully-connected Layers
        out = torch.cat([out_v, out_h], 1)
        # apply dropout
        out = self.dropout(out)

        # fully-connected layer
        z = self.ac_fc(self.fc1(out))
        x = torch.cat([z, user_emb], 1)

        w2 = self.W2(item_var)
        b2 = self.b2(item_var)

        if for_pred:
            w2 = w2.squeeze()
            b2 = b2.squeeze()
            res = (x * w2).sum(1) + b2
        else:
            res = torch.baddbmm(b2, w2, x.unsqueeze(2)).squeeze()

        return res

In [ ]:
class Recommender(object):
    """
    Contains attributes and methods that needed to train a sequential
    recommendation model. Models are trained by many tuples of
    (users, sequences, targets, negatives) and negatives are from negative
    sampling: for any known tuple of (user, sequence, targets), one or more
    items are randomly sampled to act as negatives.
    Parameters
    ----------
    n_iter: int,
        Number of iterations to run.
    batch_size: int,
        Minibatch size.
    l2: float,
        L2 loss penalty, also known as the 'lambda' of l2 regularization.
    neg_samples: int,
        Number of negative samples to generate for each targets.
        If targets=3 and neg_samples=3, then it will sample 9 negatives.
    learning_rate: float,
        Initial learning rate.
    use_cuda: boolean,
        Run the model on a GPU or CPU.
    model_args: args,
        Model-related arguments, like latent dimensions.
    """

    def __init__(self,
                 n_iter=None,
                 batch_size=None,
                 l2=None,
                 neg_samples=None,
                 learning_rate=None,
                 use_cuda=False,
                 model_args=None):

        # model related
        self._num_items = None
        self._num_users = None
        self._net = None
        self.model_args = model_args

        # learning related
        self._batch_size = batch_size
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._l2 = l2
        self._neg_samples = neg_samples
        self._device = torch.device("cuda" if use_cuda else "cpu")

        # rank evaluation related
        self.test_sequence = None
        self._candidate = dict()

    @property
    def _initialized(self):
        return self._net is not None

    def _initialize(self, interactions):
        self._num_items = interactions.num_items
        self._num_users = interactions.num_users

        self.test_sequence = interactions.test_sequences

        self._net = Caser(self._num_users,
                          self._num_items,
                          self.model_args).to(self._device)

        self._optimizer = optim.Adam(self._net.parameters(),
                                     weight_decay=self._l2,
                                     lr=self._learning_rate)

    def fit_no_train(self, train):
      """
      Does the same thing as the below fit function so the model can be loaded
      in with save weights. Will not retrain the model
      ----------
      train: :class:`spotlight.interactions.Interactions`
          training instances, also contains test sequences
      """

      if not self._initialized:
          self._initialize(train)
          
    def fit(self, train, test, verbose=False):
        """
        The general training loop to fit the model
        Parameters
        ----------
        train: :class:`spotlight.interactions.Interactions`
            training instances, also contains test sequences
        test: :class:`spotlight.interactions.Interactions`
            only contains targets for test sequences
        verbose: bool, optional
            print the logs
        """

        # convert to sequences, targets and users
        sequences_np = train.sequences.sequences
        targets_np = train.sequences.targets
        users_np = train.sequences.user_ids.reshape(-1, 1)

        L, T = train.sequences.L, train.sequences.T

        n_train = sequences_np.shape[0]

        output_str = 'total training instances: %d' % n_train
        print(output_str)

        if not self._initialized:
            self._initialize(train)

        start_epoch = 0

        for epoch_num in range(start_epoch, self._n_iter):

            t1 = time()

            # set model to training mode
            self._net.train()

            users_np, sequences_np, targets_np = shuffle(users_np,
                                                         sequences_np,
                                                         targets_np)

            negatives_np = self._generate_negative_samples(users_np, train, n=self._neg_samples)

            # convert numpy arrays to PyTorch tensors and move it to the corresponding devices
            users, sequences, targets, negatives = (torch.from_numpy(users_np).long(),
                                                    torch.from_numpy(sequences_np).long(),
                                                    torch.from_numpy(targets_np).long(),
                                                    torch.from_numpy(negatives_np).long())

            users, sequences, targets, negatives = (users.to(self._device),
                                                    sequences.to(self._device),
                                                    targets.to(self._device),
                                                    negatives.to(self._device))

            epoch_loss = 0.0

            for (minibatch_num,
                 (batch_users,
                  batch_sequences,
                  batch_targets,
                  batch_negatives)) in enumerate(minibatch(users,
                                                           sequences,
                                                           targets,
                                                           negatives,
                                                           batch_size=self._batch_size)):
                items_to_predict = torch.cat((batch_targets, batch_negatives), 1)
                items_prediction = self._net(batch_sequences,
                                             batch_users,
                                             items_to_predict)
                (targets_prediction,
                 negatives_prediction) = torch.split(items_prediction,
                                                     [batch_targets.size(1),
                                                      batch_negatives.size(1)], dim=1)

                self._optimizer.zero_grad()
                # compute the binary cross-entropy loss
                positive_loss = -torch.mean(
                    torch.log(torch.sigmoid(targets_prediction)))
                negative_loss = -torch.mean(
                    torch.log(1 - torch.sigmoid(negatives_prediction)))
                loss = positive_loss + negative_loss

                epoch_loss += loss.item()

                loss.backward()
                self._optimizer.step()

            epoch_loss /= minibatch_num + 1

            t2 = time()
            if verbose and (epoch_num + 1) % 10 == 0:
                precision, recall, mean_aps = evaluate_ranking(self, test, train, k=[1, 5, 10])
                output_str = "Epoch %d [%.1f s]\tloss=%.4f, map=%.4f, " \
                             "prec@1=%.4f, prec@5=%.4f, prec@10=%.4f, " \
                             "recall@1=%.4f, recall@5=%.4f, recall@10=%.4f, [%.1f s]" % (epoch_num + 1,
                                                                                         t2 - t1,
                                                                                         epoch_loss,
                                                                                         mean_aps,
                                                                                         np.mean(precision[0]),
                                                                                         np.mean(precision[1]),
                                                                                         np.mean(precision[2]),
                                                                                         np.mean(recall[0]),
                                                                                         np.mean(recall[1]),
                                                                                         np.mean(recall[2]),
                                                                                         time() - t2)
                print(output_str)
            else:
                output_str = "Epoch %d [%.1f s]\tloss=%.4f [%.1f s]" % (epoch_num + 1,
                                                                        t2 - t1,
                                                                        epoch_loss,
                                                                        time() - t2)
                print(output_str)

    def _generate_negative_samples(self, users, interactions, n):
        """
        Sample negative from a candidate set of each user. The
        candidate set of each user is defined by:
        {All Items} \ {Items Rated by User}
        Parameters
        ----------
        users: array of np.int64
            sequence users
        interactions: :class:`spotlight.interactions.Interactions`
            training instances, used for generate candidates
        n: int
            total number of negatives to sample for each sequence
        """

        users_ = users.squeeze()
        negative_samples = np.zeros((users_.shape[0], n), np.int64)
        if not self._candidate:
            all_items = np.arange(interactions.num_items - 1) + 1  # 0 for padding
            train = interactions.tocsr()
            for user, row in enumerate(train):
                self._candidate[user] = list(set(all_items) - set(row.indices))

        for i, u in enumerate(users_):
            for j in range(n):
                x = self._candidate[u]
                negative_samples[i, j] = x[
                    np.random.randint(len(x))]

        return negative_samples

    def predict(self, user_id, item_ids=None):
        """
        Make predictions for evaluation: given a user id, it will
        first retrieve the test sequence associated with that user
        and compute the recommendation scores for items.
        Parameters
        ----------
        user_id: int
           users id for which prediction scores needed.
        item_ids: array, optional
            Array containing the item ids for which prediction scores
            are desired. If not supplied, predictions for all items
            will be computed.
        """

        if self.test_sequence is None:
            raise ValueError('Missing test sequences, cannot make predictions')

        # set model to evaluation model
        self._net.eval()
        with torch.no_grad():
            sequences_np = self.test_sequence.sequences[user_id, :]
            sequences_np = np.atleast_2d(sequences_np)

            if item_ids is None:
                item_ids = np.arange(self._num_items).reshape(-1, 1)

            sequences = torch.from_numpy(sequences_np).long()
            item_ids = torch.from_numpy(item_ids).long()
            user_id = torch.from_numpy(np.array([[user_id]])).long()

            user, sequences, items = (user_id.to(self._device),
                                      sequences.to(self._device),
                                      item_ids.to(self._device))

            out = self._net(sequences,
                            user,
                            items,
                            for_pred=True)

        return out.cpu().numpy().flatten()




In [ ]:
# number of iterations
n_iter = 25
# seed
seed = 1234
# batch size
batch_size = 512
# learning rate
learning_rate = 1e-3
# l2 regularization
l2 = 1e-6
# negative samples
neg_samples = 3

def str2bool(v):
    return v.lower() in ('true')

use_cuda = True

# model dependent arguments
# model_arser
model_parser = dict()
model_parser['d'] = 50
model_parser['nv'] = 4
model_parser['nh'] =  16
model_parser['drop'] = 0.5
model_parser['ac_conv'] = 'relu'
model_parser['ac_fc'] ='relu'

model_config = model_parser
model_parser['L'] = L

# set seed
set_seed(seed,
          cuda=use_cuda)

# load dataset
train = Interactions(train_root)
# transform triplets to sequence representation
train.to_sequence(L, T)

test = Interactions(test_root,
                    user_map=train.user_map,
                    item_map=train.item_map)





In [ ]:
# fit model
# model = Recommender(n_iter=n_iter,
#                     batch_size=batch_size,
#                     learning_rate=learning_rate,
#                     l2=l2,
#                     neg_samples=neg_samples,
#                     model_args=model_config,
#                     use_cuda=use_cuda)
# model.fit(train, test, verbose=False)

In [ ]:
# torch.save(model._net.state_dict(), path_to_model)

In [ ]:
# Evaluate Precision@k, Recall@k, and NDCG@k

In [ ]:
# Load in model
model = Recommender(n_iter=n_iter,
                    batch_size=batch_size,
                    learning_rate=learning_rate,
                    l2=l2,
                    neg_samples=neg_samples,
                    model_args=model_config,
                    use_cuda=use_cuda)
model.fit_no_train(train)
model._net.load_state_dict(torch.load(path_to_model))
model._net.eval()

Caser(
  (user_embeddings): Embedding(5786, 50)
  (item_embeddings): Embedding(3695, 50)
  (conv_v): Conv2d(1, 4, kernel_size=(5, 1), stride=(1, 1))
  (conv_h): ModuleList(
    (0): Conv2d(1, 16, kernel_size=(1, 50), stride=(1, 1))
    (1): Conv2d(1, 16, kernel_size=(2, 50), stride=(1, 1))
    (2): Conv2d(1, 16, kernel_size=(3, 50), stride=(1, 1))
    (3): Conv2d(1, 16, kernel_size=(4, 50), stride=(1, 1))
    (4): Conv2d(1, 16, kernel_size=(5, 50), stride=(1, 1))
  )
  (fc1): Linear(in_features=280, out_features=50, bias=True)
  (W2): Embedding(3695, 100)
  (b2): Embedding(3695, 1)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
# https://github.com/dkaterenchuk/ranking_measures/blob/master/measures.py
def find_dcg(element_list):
    """
    Discounted Cumulative Gain (DCG)
    The definition of DCG can be found in this paper:
        Azzah Al-Maskari, Mark Sanderson, and Paul Clough. 2007.
        "The relationship between IR effectiveness measures and user satisfaction."
    Parameters:
        element_list - a list of ranks Ex: [5,4,2,2,1]
    Returns:
        score
    """
    score = 0.0
    for order, rank in enumerate(element_list):
        score += float(rank)/math.log((order+2))
    return score


def find_ndcg(reference, hypothesis):
    """
    Normalized Discounted Cumulative Gain (nDCG)
    Normalized version of DCG:
        nDCG = DCG(hypothesis)/DCG(reference)
    Parameters:
        reference   - a gold standard (perfect) ordering Ex: [5,4,3,2,1]
        hypothesis  - a proposed ordering Ex: [5,2,2,3,1]
    Returns:
        ndcg_score  - normalized score
    """

    return find_dcg(hypothesis)/find_dcg(reference)

def def_value():
  return 0

In [ ]:
def compute_precision_recall_ndcg(targets, predictions, k):
    pred = predictions[:k]
    targ = targets[:k]
    num_hit = len(set(pred).intersection(set(targ)))

    # Compute precision and recall
    precision = float(num_hit) / len(pred)
    recall = float(num_hit) / len(targ)

    # Compute NDCG
    # Relevance scores in ideal order
    true_relevance = []
          
    d = defaultdict(def_value)

    for i in range(len(targ), 0, -1):
      true_relevance.append(i)

    # Relevance scores in output order
    for i, value in enumerate(true_relevance):
      d[targ[i]] = value
    
    pred_relevance = []

    for predicted in pred:
      pred_relevance.append(d[predicted])
    
    # Normalized DCG score
    ndcg = find_ndcg(true_relevance, pred_relevance)

    return precision, recall, ndcg

In [ ]:
test = test.tocsr()
k = [5, 10, 15]

if train is not None:
    train = train.tocsr()

precisions = [list() for _ in range(len(k))]
recalls = [list() for _ in range(len(k))]
NDCGs = [list() for _ in range(len(k))]

for user_id, row in enumerate(test):
    if not len(row.indices):
        continue

    predictions = -model.predict(user_id)
    predictions = predictions.argsort()

    if train is not None:
        rated = set(train[user_id].indices)
    else:
        rated = []

    predictions = [p for p in predictions if p not in rated]

    targets = row.indices

    for i, _k in enumerate(k):
        precision, recall, NDCG = compute_precision_recall_ndcg(targets, predictions, _k)
        precisions[i].append(precision)
        recalls[i].append(recall)
        NDCGs[i].append(NDCG)

precisions = [np.mean(i) for i in precisions]
recalls = [np.mean(i) for i in recalls]
NDCGs = [np.mean(i) for i in NDCGs]

if not isinstance(k, list):
    precisions = precisions[0]
    recalls = recalls[0]
    NDCGs = NDCGs[0]

In [ ]:
# Writing to file
with open(path_to_txt, "w") as file1:
    # Writing data to a file
    for i, _k in enumerate(k):
      file1.write('Precision at {} = {}\n'.format(_k, precisions[i]))
      file1.write('Recall at {} = {}\n'.format(_k, recalls[i]))
      file1.write('NDCG at {} = {}\n'.format(_k, NDCGs[i]))